# TEI XML to Interactive HTML Converter

This notebook converts a parallel Greek/Latin grammar from its TEI XML format into a standalone, interactive HTML file.

### Features:
* **Two-Column Layout:** Displays the Greek and Latin texts side-by-side for easy comparison.
* **Interactive Highlighting:** Hovering over a numbered section in one language will highlight the corresponding section in the other language.
* **Semantic Styling:** Grammatical terms, citations, quotes, and translations are styled for readability.
* **Standalone File:** All CSS and JavaScript are embedded directly into the HTML file, making it fully portable.

### Instructions:
1.  Make sure the source file `russell1902-comp-greek-latin-grammar.xml` is in the same directory as this notebook.
2.  Run all the cells in order.
3.  A link to the generated `russell_grammar.html` file will appear at the end.

In [199]:
import os
import re
import json
from lxml import etree
from IPython.display import HTML, FileLink

print("Libraries imported successfully.")

Libraries imported successfully.


In [200]:
# All CSS and JavaScript will be embedded in the final HTML for portability.

CSS_STYLES = """
<style>
    body {
        font-family: 'Georgia', serif; line-height: 1.6; color: #333;
        margin: 0; padding: 2em; background-color: #fdfdfd;
        scroll-padding-top: 1em;
    }
    .container { max-width: 1600px; margin: 0 auto; }
    h1, h2, h3 {
        font-family: 'Helvetica Neue', sans-serif; color: #2c3e50;
        border-bottom: 2px solid #e0e0e0; padding-bottom: 5px; margin-top: 1.5em;
    }
    .grid-container {
        display: grid; grid-template-columns: 1fr 1fr; gap: 30px;
        align-items: start; margin-bottom: 2em; border-bottom: 1px solid #eee; padding-bottom: 2em;
    }
    .full-width-section { margin-bottom: 2em; padding-bottom: 2em; border-bottom: 1px solid #eee; }
    .latin-col, .greek-col, .english-col {
        padding: 15px; border: 1px solid #ddd; background-color: #fff;
        border-radius: 5px; min-height: 50px;
        max-height: 85vh; overflow-y: auto; scroll-padding-top: 1em;
    }
    div.section, div.note, div.observation, div.excursus, div.form_division, div.textpart {
        border: 1px solid #f0f0f0; border-left: 4px solid #3498db;
        padding: 10px 15px; margin-bottom: 1em; border-radius: 4px;
        transition: background-color 0.2s ease-in-out; overflow: auto;
        cursor: pointer;
    }
    .section-anchor {
        font-family: 'Helvetica Neue', sans-serif; font-weight: bold; color: #95a5a6;
        text-decoration: none; margin-right: 1em; float: left; padding: 0.1em 0.5em;
        border: 1px solid #e0e0e0; border-radius: 4px; background-color: #f8f9f9;
    }
    .section-anchor:hover { color: #fff; background-color: #3498db; border-color: #2980b9; }
    .section.empty { border: 1px dashed #e0e0e0; background-color: #fafafa; min-height: 30px; }
    div.note { border-left-color: #f1c40f; background-color: #fef9e7; }
    div.excursus { border-left-color: #9b59b6; }
    .highlight { background-color: #eaf2f8 !important; box-shadow: 0 0 10px rgba(52, 152, 219, 0.5); }
    .pb { display: block; text-align: right; font-style: italic; color: #95a5a6; margin: 1em 0; font-size: 0.9em; }
    .pb::before { content: 'p. ' attr(data-n); }
    term, .term { font-style: italic; color: #2980b9; }
    q, .q { font-style: italic; }
    cit, blockquote { margin: 1em 20px; padding: 10px; border-left: 3px solid #bdc3c7; background-color: #f8f9f9; }
    bibl, cite { display: block; text-align: right; font-weight: bold; font-style: normal; color: #7f8c8d; }
    .translation { font-style: italic; color: #566573; margin-top: 5px; }
    foreign, .foreign { font-family: monospace; background-color: #ecf0f1; padding: 1px 4px; border-radius: 3px; }
    list { padding-left: 20px; }
    #toc-sidebar {
        position: fixed; left: 0; top: 0; height: 100vh; width: 350px;
        background-color: #f8f9f9; border-right: 1px solid #e0e0e0;
        padding: 20px; overflow-y: auto;
        transform: translateX(-100%); transition: transform 0.3s ease-in-out;
        z-index: 1000; box-sizing: border-box;
    }
    #toc-sidebar.open { transform: translateX(0); }
    #toc-sidebar h2 { margin-top: 0; border-bottom: 2px solid #3498db; }
    #toc-sidebar ul { list-style: none; padding-left: 10px; }
    #toc-sidebar li { margin-bottom: 0.5em; font-size: 0.9em; }
    #toc-sidebar a { text-decoration: none; color: #2c3e50; transition: color 0.2s; }
    #toc-sidebar a:hover { color: #3498db; }
    .toc-label { font-weight: bold; color: #7f8c8d; margin-right: 5px; }
    #main-content { transition: margin-left 0.3s ease-in-out; }
    #main-content.toc-open { margin-left: 350px; }
    #toc-toggle {
        position: fixed; top: 15px; left: 15px; z-index: 1001;
        cursor: pointer; transition: left 0.3s ease-in-out;
        font-size: 1.5em; padding: 5px 15px; background-color: #3498db;
        color: white; border: 1px solid #2980b9; border-radius: 4px; box-shadow: 0 2px 5px rgba(0,0,0,0.2);
    }
    #toc-toggle.toc-open { left: 365px; }
</style>
"""

JS_INTERACTIVITY = """
<script>
    document.addEventListener('DOMContentLoaded', function() {
        const links = LINK_DATA;
        function highlightPartners(element, add) {
            const currentId = '#' + element.id;
            const partnerIds = links[currentId];
            const action = add ? 'add' : 'remove';
            if (element) element.classList[action]('highlight');
            if (partnerIds) {
                partnerIds.forEach(partnerId => {
                    const partnerEl = document.querySelector(partnerId);
                    if (partnerEl) partnerEl.classList[action]('highlight');
                });
            }
        }
        document.querySelectorAll('[id]').forEach(el => {
            const elId = '#' + el.id;
            if (links[elId]) {
                 el.addEventListener('mouseover', () => highlightPartners(el, true));
                 el.addEventListener('mouseout', () => highlightPartners(el, false));
                 el.addEventListener('click', (event) => {
                    if (event.target.closest('a')) {
                        event.preventDefault();
                    }
                    el.scrollIntoView({ behavior: 'smooth', block: 'start' });
                    const partnerIds = links[elId];
                    if (partnerIds) {
                        partnerIds.forEach(partnerId => {
                            const partnerEl = document.querySelector(partnerId);
                            if (partnerEl) {
                                partnerEl.scrollIntoView({ behavior: 'smooth', block: 'start' });
                            }
                        });
                    }
                 });
            }
        });
        const toggleBtn = document.getElementById('toc-toggle');
        const sidebar = document.getElementById('toc-sidebar');
        const mainContent = document.getElementById('main-content');
        if (toggleBtn && sidebar && mainContent) {
            toggleBtn.addEventListener('click', () => {
                sidebar.classList.toggle('open');
                mainContent.classList.toggle('toc-open');
                toggleBtn.classList.toggle('toc-open');
                toggleBtn.innerHTML = sidebar.classList.contains('open') ? '&times;' : '&#9776;';
            });
        }
        document.querySelectorAll('#toc-sidebar a').forEach(link => {
            link.addEventListener('click', function (event) {
                event.preventDefault();
                const targetId = this.getAttribute('href');
                const targetElement = document.querySelector(targetId);
                if (targetElement) {
                    targetElement.scrollIntoView({ behavior: 'smooth', block: 'start' });
                    const partnerIds = links[targetId];
                    if (partnerIds) {
                        partnerIds.forEach(partnerId => {
                            const partnerEl = document.querySelector(partnerId);
                            if (partnerEl) partnerEl.scrollIntoView({ behavior: 'smooth', block: 'start' });
                        });
                    }
                }
                if (sidebar.classList.contains('open')) {
                    toggleBtn.click();
                }
            });
        });

        // --- UPDATED: LOGIC FOR URL HASH ALIGNMENT ---
        function alignPanesFromURL() {
            const hash = window.location.hash;
            if (hash && links[hash]) {
                const partnerIds = links[hash];
                partnerIds.forEach(partnerId => {
                    const partnerEl = document.querySelector(partnerId);
                    if (partnerEl) {
                        partnerEl.scrollIntoView({ behavior: 'smooth', block: 'start' });
                    }
                });
                const primaryEl = document.querySelector(hash);
                if (primaryEl) {
                    primaryEl.scrollIntoView({ behavior: 'smooth', block: 'start' });
                }
            }
        }

        // Listen for when the URL hash changes.
        window.addEventListener('hashchange', alignPanesFromURL, false);

        // Run the alignment function on initial page load after a brief delay
        // to prevent a conflict with the browser's native scroll behavior.
        window.addEventListener('load', () => {
            setTimeout(alignPanesFromURL, 100);
        });
        // --- END UPDATE ---
    });
</script>
"""

In [201]:
XML_FILE = '../GRC_misc/russell1902-comp-greek-latin-grammar.xml'
HTML_FILE = 'russell_grammar.html'
root = None

try:
    parser = etree.XMLParser(remove_blank_text=False)
    tree = etree.parse(XML_FILE, parser)
    root = tree.getroot()
    print(f"Successfully parsed '{XML_FILE}'.")

    # --- FIX: Store plain IDs without '#' for reliable JavaScript lookup ---
    link_map = {}
    for link in root.findall('.//linkGrp/link'):
        # Get raw IDs and strip the '#' prefix from each
        targets = [t.lstrip('#') for t in link.get('target', '').split() if t]
        for i in range(len(targets)):
            for j in range(i + 1, len(targets)):
                id1, id2 = targets[i], targets[j]
                link_map.setdefault(id1, []).append(id2)
                link_map.setdefault(id2, []).append(id1)

    print(f"Created simplified link map with {len(link_map)} linked elements.")

except IOError:
    print(f"Error: Could not find the file '{XML_FILE}'.")
    root = None
except Exception as e:
    print(f"An error occurred: {e}")

Successfully parsed '../GRC_misc/russell1902-comp-greek-latin-grammar.xml'.
Created simplified link map with 1652 linked elements.


In [202]:
def transform_element_to_html(element):
    """
    Recursively transforms a TEI element and its children into an HTML string.
    """
    if not isinstance(element.tag, str): return "" # Filter out comments
    
    tag = etree.QName(element.tag).localname
    html_content = ""
    
    if tag == 'div' and element.get('type') in ('section', 'form_division', 'textpart'):
        n_val = element.get("n")
        xml_id = element.get('{http://www.w3.org/XML/1998/namespace}id')
        if n_val and xml_id:
            html_content += f'<a class="section-anchor" href="#{xml_id}">{n_val}</a>'
            
    if element.text:
        html_content += element.text.replace('&', '&amp;')

    tag_map = {
        'div': 'div', 'p': 'p', 'head': 'h2', 'term': 'span', 'foreign': 'span', 
        'list': 'ul', 'item': 'li', 'q': 'q', 'cit': 'blockquote', 'bibl': 'cite', 'label': 'span'
    }
    html_tag = tag_map.get(tag, 'span')

    if tag in ('pb', 'fw', 'milestone'): return ""
    
    if tag == 'quote':
        quote_text = (element.text or '').replace('&', '&amp;')
        return f'<p class="translation">{quote_text}</p>' if element.get('type') == 'translation' else f'<p class="quote">{quote_text}</p>'

    if tag == 'ref':
        target = element.get('target', '').split('-')[0]
        return f'<a href="#{target}">{element.text or ""}</a>'

    attrs = ''
    if '{http://www.w3.org/XML/1998/namespace}id' in element.attrib:
        attrs += f' id="{element.attrib["{http://www.w3.org/XML/1998/namespace}id"]}"'
        
    class_list = [tag]
    if element.get('type'): class_list.append(element.get('type'))
    if element.get('subtype'): class_list.append(element.get('subtype'))
    if tag == 'label': class_list.append('toc-label')
    if element.get('n'): attrs += f' data-n="{element.get("n")}"'
    if tag == 'div' and not (element.text or list(element)): class_list.append('empty')
    attrs += f' class="{" ".join(class_list)}"'
    
    for child in element:
        html_content += transform_element_to_html(child)
        if child.tail: html_content += child.tail.replace('&', '&amp;')
            
    return f'<{html_tag}{attrs}>{html_content}</{html_tag}>'


def generate_toc_html(root):
    """
    Parses the <teiHeader> and <div type="contents"> to generate the TOC sidebar HTML.
    """
    # Define all header information within this dictionary
    header_info = {
        'title': root.findtext('.//teiHeader/fileDesc/titleStmt/title', 'Title Not Found'),
        'author': root.findtext('.//teiHeader/fileDesc/titleStmt/author', 'Author Not Found'),
        'publisher': root.findtext('.//teiHeader/fileDesc/publicationStmt/publisher', ''),
        'pubPlace': root.findtext('.//teiHeader/fileDesc/publicationStmt/pubPlace', ''),
        'date': root.findtext('.//teiHeader/fileDesc/publicationStmt/date', ''),
        # The XPath for the source URL is correctly placed here
        'source_url': root.findtext('.//teiHeader/fileDesc/sourceDesc/biblStruct/monogr/idno[@type="hathitrust"]', '#')
    }

    # Build the HTML string using the dictionary variables
    header_html = f"""
    <div class="toc-header">
        <h3>{header_info['title']}</h3>
        <p><strong>Author:</strong> {header_info['author']}</p>
        <p>{header_info['publisher']}, {header_info['pubPlace']}, {header_info['date']}</p>
        <p>View Source Scan on HathiTrust <a href="{header_info['source_url']}" target="_blank" rel="noopener noreferrer">{header_info['source_url']}</a></p>
    </div>
    """
    
    contents_div = root.find('.//div[@type="contents"]')
    
    # FIX: This 'if' statement is now correctly unindented
    if contents_div is None:
        return header_html

    toc_list_html = "<h2>Contents</h2>"
    
    def process_list(element):
        list_html = "<ul>"
        for item in element.iterchildren('item'):
            if not isinstance(item.tag, str): continue
            
            item_text_parts = []
            label = item.find('label')
            if label is not None and label.text:
                item_text_parts.append(f"<span class='toc-label'>{label.text.strip()}</span>")

            if item.text: item_text_parts.append(item.text.strip())

            for child in item:
                tag = etree.QName(child.tag).localname if isinstance(child.tag, str) else ''
                if tag not in ['list', 'milestone', 'label'] and child.text:
                    item_text_parts.append(child.text.strip())
                if child.tail:
                    item_text_parts.append(child.tail.strip())

            item_text = ' '.join(filter(None, item_text_parts))
            item_text = re.sub(r'\s+', ' ', item_text).strip()
            item_text = re.sub(r'\s+\(', '(', item_text)
            item_text = re.sub(r'\s+\)', ')', item_text)
            item_text = re.sub(r'\s+;', ';', item_text)
            
            ref_tag = item.find('.//ref')
            # Get the plain ID without the '#'
            href = f'#{ref_tag.get("target", "").split("-")[0]}' if ref_tag is not None else '#'
            
            list_html += f'<li><a href="{href}">{item_text}</a>'
            
            nested_list = item.find('list')
            if nested_list is not None:
                list_html += process_list(nested_list)
            list_html += '</li>'
            
        return list_html + "</ul>"

    toc_list = contents_div.find('list')
    if toc_list is not None:
        toc_list_html += process_list(toc_list)
        
    return header_html + toc_list_html
# --- NEW: Added styling for the new .toc-header element ---
CSS_STYLES = """
<style>
    body {
        font-family: 'Georgia', serif; line-height: 1.6; color: #333;
        margin: 0; padding: 2em; background-color: #fdfdfd;
        scroll-padding-top: 1em;
    }
    .container { max-width: 1600px; margin: 0 auto; }
    h1, h2, h3 {
        font-family: 'Helvetica Neue', sans-serif; color: #2c3e50;
        border-bottom: 2px solid #e0e0e0; padding-bottom: 5px; margin-top: 1.5em;
    }
    .grid-container {
        display: grid; grid-template-columns: 1fr 1fr; gap: 30px;
        align-items: start; margin-bottom: 2em; border-bottom: 1px solid #eee; padding-bottom: 2em;
    }
    .full-width-section { margin-bottom: 2em; padding-bottom: 2em; border-bottom: 1px solid #eee; }
    .latin-col, .greek-col, .english-col {
        padding: 15px; border: 1px solid #ddd; background-color: #fff;
        border-radius: 5px; min-height: 50px;
        max-height: 85vh; overflow-y: auto; scroll-padding-top: 1em;
    }
    div.section, div.note, div.observation, div.excursus, div.form_division, div.textpart {
        border: 1px solid #f0f0f0; border-left: 4px solid #3498db;
        padding: 10px 15px; margin-bottom: 1em; border-radius: 4px;
        transition: background-color 0.2s ease-in-out; overflow: auto;
        cursor: pointer;
    }
    .section-anchor {
        font-family: 'Helvetica Neue', sans-serif; font-weight: bold; color: #95a5a6;
        text-decoration: none; margin-right: 1em; float: left; padding: 0.1em 0.5em;
        border: 1px solid #e0e0e0; border-radius: 4px; background-color: #f8f9f9;
    }
    .section-anchor:hover { color: #fff; background-color: #3498db; border-color: #2980b9; }
    .section.empty { border: 1px dashed #e0e0e0; background-color: #fafafa; min-height: 30px; }
    div.note { border-left-color: #f1c40f; background-color: #fef9e7; }
    div.excursus { border-left-color: #9b59b6; }
    .highlight { background-color: #eaf2f8 !important; box-shadow: 0 0 10px rgba(52, 152, 219, 0.5); }
    .pb { display: block; text-align: right; font-style: italic; color: #95a5a6; margin: 1em 0; font-size: 0.9em; }
    .pb::before { content: 'p. ' attr(data-n); }
    term, .term { font-style: italic; color: #2980b9; }
    q, .q { font-style: italic; }
    cit, blockquote { margin: 1em 20px; padding: 10px; border-left: 3px solid #bdc3c7; background-color: #f8f9f9; }
    bibl, cite { display: block; text-align: right; font-weight: bold; font-style: normal; color: #7f8c8d; }
    .translation { font-style: italic; color: #566573; margin-top: 5px; }
    foreign, .foreign { font-family: monospace; background-color: #ecf0f1; padding: 1px 4px; border-radius: 3px; }
    list { padding-left: 20px; }
    #toc-sidebar {
        position: fixed; left: 0; top: 0; height: 100vh; width: 350px;
        background-color: #f8f9f9; border-right: 1px solid #e0e0e0;
        padding: 20px; overflow-y: auto;
        transform: translateX(-100%); transition: transform 0.3s ease-in-out;
        z-index: 1000; box-sizing: border-box;
    }
    #toc-sidebar.open { transform: translateX(0); }
    #toc-sidebar h2 { margin-top: 0.5em; border-bottom: 2px solid #3498db; }
    .toc-header {
        margin-bottom: 1em;
        padding-bottom: 0.5em;
        border-bottom: 1px solid #ccc;
    }
    .toc-header h3 {
        margin-top: 0; margin-bottom: 0.5em; font-size: 1em;
        color: #2c3e50; border-bottom: none;
    }
    .toc-header p {
        font-size: 0.8em; margin: 0.3em 0; color: #566573;
    }
    .toc-header p a { font-weight: bold; }
    #toc-sidebar ul { list-style: none; padding-left: 10px; }
    #toc-sidebar li { margin-bottom: 0.5em; font-size: 0.9em; }
    #toc-sidebar a { text-decoration: none; color: #2c3e50; transition: color 0.2s; }
    #toc-sidebar a:hover { color: #3498db; }
    .toc-label { font-weight: bold; color: #7f8c8d; margin-right: 5px; }
    #main-content { transition: margin-left 0.3s ease-in-out; }
    #main-content.toc-open { margin-left: 350px; }
    #toc-toggle {
        position: fixed; top: 15px; left: 15px; z-index: 1001;
        cursor: pointer; transition: left 0.3s ease-in-out;
        font-size: 1.5em; padding: 5px 15px; background-color: #3498db;
        color: white; border: 1px solid #2980b9; border-radius: 4px; box-shadow: 0 2px 5px rgba(0,0,0,0.2);
    }
    #toc-toggle.toc-open { left: 365px; }
</style>
"""

print("Functions and styles updated with TOC header information. Please run the final cell to generate the HTML.")

Functions and styles updated with TOC header information. Please run the final cell to generate the HTML.


In [203]:
if root is not None:
    body = root.find('.//text/body')
    if body is not None:
        toc_html = generate_toc_html(root)
        
# --- FIX: This version uses .format() to avoid the f-string error and corrects the link-blocking logic ---
        JS_INTERACTIVITY_FIXED = """
<script>
    document.addEventListener('DOMContentLoaded', function() {{
        const links = {link_data};

        function scrollToElementInPane(element) {{
            if (!element) return;
            const scrollableParent = element.closest('.latin-col, .greek-col, .english-col');
            
            if (scrollableParent) {{
                const gridContainer = scrollableParent.closest('.grid-container');
                if (gridContainer) {{
                    const rect = gridContainer.getBoundingClientRect();
                    if (rect.top < 0 || rect.top > window.innerHeight * 0.8) {{
                       gridContainer.scrollIntoView({{ behavior: 'smooth', block: 'start' }});
                    }}
                }}
                setTimeout(() => {{
                    const offset = 10;
                    const scrollTop = element.offsetTop - scrollableParent.offsetTop - offset;
                    scrollableParent.scrollTo({{ top: scrollTop, behavior: 'smooth' }});
                }}, 200);

            }} else {{
                element.scrollIntoView({{ behavior: 'smooth', block: 'start' }});
            }}
        }}

        function scrollAllPartners(elementId) {{
            if (!elementId) return;
            const mainElement = document.getElementById(elementId);
            if (mainElement) {{
                scrollToElementInPane(mainElement);
            }}
            const partnerIds = links[elementId];
            if (partnerIds) {{
                partnerIds.forEach(partnerId => {{
                    const partnerEl = document.getElementById(partnerId);
                    if (partnerEl) scrollToElementInPane(partnerEl);
                }});
            }}
        }}
        
        function highlightPartners(element, add) {{
            const partnerIds = links[element.id];
            const action = add ? 'add' : 'remove';
            if (element) element.classList[action]('highlight');
            if (partnerIds) {{
                partnerIds.forEach(partnerId => {{
                    const partnerEl = document.getElementById(partnerId);
                    if (partnerEl) partnerEl.classList[action]('highlight');
                }});
            }}
        }}

        document.querySelectorAll('[id]').forEach(el => {{
            if (links[el.id]) {{
                 el.addEventListener('mouseover', () => highlightPartners(el, true));
                 el.addEventListener('mouseout', () => highlightPartners(el, false));
                 el.addEventListener('click', (event) => {{
                    event.preventDefault();
                    const anchor = event.target.closest('a[href^="#"]');
                    if (anchor && anchor.getAttribute('href').length > 1) {{
                        scrollAllPartners(anchor.getAttribute('href').substring(1));
                    }} else {{
                        scrollAllPartners(el.id);
                    }}
                 }});
            }}
        }});

        const toggleBtn = document.getElementById('toc-toggle');
        const sidebar = document.getElementById('toc-sidebar');
        const mainContent = document.getElementById('main-content');
        if (toggleBtn && sidebar && mainContent) {{
            toggleBtn.addEventListener('click', () => {{
                sidebar.classList.toggle('open');
                mainContent.classList.toggle('toc-open');
                toggleBtn.classList.toggle('toc-open');
                toggleBtn.innerHTML = sidebar.classList.contains('open') ? '&times;' : '&#9776;';
            }});
        }}
        
        // --- THIS IS THE CORRECTED CLICK HANDLER FOR SIDEBAR LINKS ---
        document.querySelectorAll('#toc-sidebar a').forEach(link => {{
            link.addEventListener('click', function (event) {{
                const href = this.getAttribute('href');
                // Only prevent default and scroll for internal links
                if (href && href.startsWith('#')) {{
                    event.preventDefault();
                    const targetId = href.substring(1);
                    scrollAllPartners(targetId);
                }}
                // For external links, do nothing and let the browser handle it.
            }});
        }});

        function alignPanesFromURL() {{
            if (window.location.hash) {{
                const elementId = window.location.hash.substring(1);
                scrollAllPartners(elementId);
            }}
        }}

        window.addEventListener('hashchange', alignPanesFromURL, false);
        window.addEventListener('load', () => {{
            setTimeout(alignPanesFromURL, 200);
        }});
    }});
</script>
""".format(link_data=json.dumps(link_map, indent=4))
        
        html_parts = [
            f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Parallel Greek and Latin Grammar</title>
    {CSS_STYLES}
    {JS_INTERACTIVITY_FIXED}
</head>
<body>
    <button id="toc-toggle" class="toc-open">&times;</button>
    <nav id="toc-sidebar" class="open">{toc_html}</nav>
    <div id="main-content" class="toc-open">
        <div class="container">
            <h1>Parallel Greek and Latin Grammar</h1>
            <p style="background-color: #eaf2f8; border-left: 4px solid #3498db; padding: 1em;">
                <strong>Note:</strong> Click the <strong>&times;</strong> button to collapse the Table of Contents.
            </p>"""
        ]

        children = [child for child in body if child.tag is not etree.Comment]
        i = 0
        ns = {'xml': 'http://www.w3.org/XML/1998/namespace'}
        
        while i < len(children):
            child = children[i]
            tag = etree.QName(child.tag).localname
            
            if tag == 'div' and child.get('type') == 'contents':
                i += 1
                continue

            is_lat_col = tag == 'div' and child.get('type') == 'column' and child.get(f'{{{ns["xml"]}}}lang') == 'lat'
            is_parallel = tag == 'div' and child.get('type') == 'parallel'

            if is_lat_col or is_parallel:
                lat_col = child if is_lat_col else child.find('./div[@xml:lang="la"]', namespaces=ns)
                grc_col = None
                eng_col_content = ""

                if is_lat_col and (i + 1) < len(children):
                    next_child = children[i+1]
                    if etree.QName(next_child.tag).localname == 'div' and next_child.get('type') == 'column' and next_child.get(f'{{{ns["xml"]}}}lang') == 'grc':
                        grc_col = next_child
                elif is_parallel:
                    eng_col = child.find('./div[@xml:lang="en"]', namespaces=ns)
                    if eng_col is not None: eng_col_content = "".join(map(transform_element_to_html, eng_col))
                    grc_col = child.find('./div[@xml:lang="grc"]', namespaces=ns)
                
                lat_html = "".join(map(transform_element_to_html, lat_col)) if lat_col is not None else '<div class="section empty"></div>'
                grc_html = "".join(map(transform_element_to_html, grc_col)) if grc_col is not None else '<div class="section empty"></div>'

                if is_parallel and eng_col_content:
                    html_parts.append(f'<style>.three-col {{ grid-template-columns: 1fr 1fr 1fr; }}</style><div class="grid-container three-col"><div class="latin-col">{lat_html}</div><div class="english-col">{eng_col_content}</div><div class="greek-col">{grc_html}</div></div>')
                else:
                    html_parts.append(f'<div class="grid-container"><div class="latin-col">{lat_html}</div><div class="greek-col">{grc_html}</div></div>')
                
                i += 2 if is_lat_col and grc_col is not None else 1
            else:
                html_parts.append(f'<div class="full-width-section">{transform_element_to_html(child)}</div>')
                i += 1
        
        html_parts.append("""
        </div>
    </div>
</body>
</html>""")
        
        final_html = "".join(html_parts)
        
        with open(HTML_FILE, 'w', encoding='utf-8') as f:
            f.write(final_html)
        
        print(f"Successfully generated '{HTML_FILE}' with corrected scroll alignment.")
        display(FileLink(HTML_FILE))

    else:
        print("Error: Could not find a <body> element.")
else:
    print("Skipping HTML generation due to XML parsing error.")

Successfully generated 'russell_grammar.html' with corrected scroll alignment.


/Users/gcrane/github/JupMisc/russell_grammar.html